In [0]:
!pip install bert-for-tf2
!pip install sentencepiece

     |████████████████████████████████| 40kB 2.1MB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.4-cp36-none-any.whl size=30114 sha256=a07030e4781b1cfebaa655ce9e69c4da62c53dda6f1c22f1a951e66735237abf
  Stored in directory: /root/.cache/pip/wheels/cf/3f/4d/79d7735015a5f523648df90d871ce8e89a7df8185f7703eeab
  Created wheel for py-params: filename=py_params-0.9.7-cp36-none-any.whl size=7302 sha256=539097d1275dd1b5e1f6563ad660b9c47e8cd88e0b528307627ad3cb0fad7332
  Stored in directory: /root/.cache/pip/wheels/67/f5/19/b461849a50aefdf4bab47c4756596e82ee2118b8278e5a1980
  Created wheel for params-flow: filename=params_flow-0.8.2-cp36-none-any.whl size=19473 sha256=a7b655bf3cf5e9f2876ee3c4106a0503155de1997e2e3a640d12cd69ba1381d5
  Stored in directory: /root/.cache/pip/wheels/08/c8/7f/81c86b9ff2b86e2c477e3914175be03e679e596067dc630c06
Successfully built bert-for-tf2 py-params params-flow
     |████████████████████████████████| 1.1MB 3.5MB/s 


In [0]:
try:
    %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf

import tensorflow_hub as hub

from tensorflow.keras import layers
import bert
import pandas as pd
import numpy as np

In [0]:
from google.colab import files
files.upload()

!mkdir ~p ~/.kaggle
!cp kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json

!kaggle competitions download -c fake-news

import zipfile
with zipfile.ZipFile('./test.csv.zip', 'r') as zip_ref:
    zip_ref.extractall('./')


with zipfile.ZipFile('./train.csv.zip', 'r') as zip_ref:
    zip_ref.extractall('./')

Saving kaggle.json to kaggle.json
 70% 26.0M/37.0M [00:00<00:00, 38.0MB/s]
100% 37.0M/37.0M [00:00<00:00, 58.4MB/s]
  0% 0.00/40.6k [00:00<?, ?B/s]
100% 40.6k/40.6k [00:00<00:00, 41.6MB/s]
 85% 8.00M/9.42M [00:00<00:00, 83.4MB/s]
100% 9.42M/9.42M [00:00<00:00, 60.2MB/s]


In [0]:
train_data=pd.read_csv("train.csv")
test_data=pd.read_csv("test.csv")
submit_data=pd.read_csv("submit.csv")

# contains train data 
train_data.head(100)

# contains test data 
test_data.head()

# contains label for test data 
submit_data.head()

,id,label
0,20800,0
1,20801,1
2,20802,0
3,20803,1
4,20804,1


In [0]:
train_data = train_data[train_data["text"].notna()]
train_data["title"].fillna(" ", inplace = True) 
train_data["author"].fillna(" ", inplace = True) 
test_data["label"]=submit_data["label"]

# Making a general function for our test set 
def clean(data):
    data = data[data["text"].notna()]
    data["title"].fillna(" ", inplace = True) 
    data["author"].fillna(" ", inplace = True)
    return data

# now we will clean test data

test_data=clean(test_data)

test_data.isna().sum()

frames = [train_data,test_data]

whole = pd.concat(frames, ignore_index=True)

whole.shape   #  (25954, 5)

train_data.shape  # (20761, 5)

whole.isna().sum()
# whole.tail(10)

whole['label'].value_counts()
# 1    13228
# 0    12726

whole['soup'] = whole['author'] +" "+ whole['title']

whole['soup'].isna().sum()

whole.head(10)

whole.drop(columns=['id','author','title','text'],inplace=True)

# whole.reset_index(inplace = True) 

whole.head(10)

,label,soup
0,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,1,Consortiumnews.com Why the Truth Might Get You...
3,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,1,Howard Portnoy Iranian woman jailed for fictio...
5,0,Daniel Nussbaum Jackie Mason: Hollywood Would ...
6,1,Life: Life Of Luxury: Elton John’s 6 Favorit...
7,0,Alissa J. Rubin Benoît Hamon Wins French Socia...
8,0,Excerpts From a Draft Script for Donald Trum...
9,0,Megan Twohey and Scott Shane A Back-Channel Pl...


In [0]:
def preprocess_text(sen):
    # Removing html tags
    sentence = remove_tags(sen)

    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence

In [0]:
import re
TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    return TAG_RE.sub('', text)

In [0]:
reviews = []
sentences = list(whole['soup'])
for sen in sentences:
    reviews.append(preprocess_text(sen))

In [0]:
print(whole.columns.values)

['label' 'soup']


In [0]:
whole.label.unique()

array([1, 0])

In [0]:
y = np.array(whole['label'])

In [0]:
BertTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=False)
vocabulary_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
to_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = BertTokenizer(vocabulary_file, to_lower_case)

In [0]:
tokenizer.tokenize("don't be so judgmental")

['don', "'", 't', 'be', 'so', 'judgment', '##al']

In [0]:
tokenizer.convert_tokens_to_ids(tokenizer.tokenize("dont be so judgmental"))

[2123, 2102, 2022, 2061, 8689, 2389]

In [0]:
def tokenize_reviews(text_reviews):
    return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text_reviews))

In [0]:
tokenized_reviews = [tokenize_reviews(review) for review in reviews]

In [0]:
reviews_with_len = [[review, y[i], len(review)]
                 for i, review in enumerate(tokenized_reviews)]

In [0]:
import random
random.shuffle(reviews_with_len)

reviews_with_len.sort(key=lambda x: x[2])

In [0]:
sorted_reviews_labels = [(review_lab[0], review_lab[1]) for review_lab in reviews_with_len]

In [0]:
processed_dataset = tf.data.Dataset.from_generator(lambda: sorted_reviews_labels, output_types=(tf.int32, tf.int32))

In [0]:
BATCH_SIZE = 32
batched_dataset = processed_dataset.padded_batch(BATCH_SIZE,padded_shapes=((None, ), ()))

In [0]:
# print the first batch and see how padding has been applied to it...
next(iter(batched_dataset))

(<tf.Tensor: shape=(32, 0), dtype=int32, numpy=array([], shape=(32, 0), dtype=int32)>,
 <tf.Tensor: shape=(32,), dtype=int32, numpy=
 array([1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)>)

In [0]:
import math

TOTAL_BATCHES = math.ceil(len(sorted_reviews_labels) / BATCH_SIZE)  # 313
TEST_BATCHES = math.floor(TOTAL_BATCHES * 0.2)      # 162
TAKE_BATCHES = TOTAL_BATCHES - TEST_BATCHES       # 650
batched_dataset.shuffle(TOTAL_BATCHES)
test_data = batched_dataset.skip(TAKE_BATCHES)
train_data = batched_dataset.skip(TEST_BATCHES)

In [0]:
class TEXT_MODEL(tf.keras.Model):
    
    def __init__(self,
                 vocabulary_size,
                 embedding_dimensions=128,
                 cnn_filters=50,
                 dnn_units=512,
                 model_output_classes=2,
                 dropout_rate=0.1,
                 training=False,
                 name="text_model"):
        super(TEXT_MODEL, self).__init__(name=name)
        
        self.embedding = layers.Embedding(vocabulary_size,
                                          embedding_dimensions)
        self.cnn_layer1 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=2,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer2 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=3,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer3 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=4,
                                        padding="valid",
                                        activation="relu")
        self.pool = layers.GlobalMaxPool1D()
        
        self.dense_1 = layers.Dense(units=dnn_units, activation="relu")
        self.dropout = layers.Dropout(rate=dropout_rate)
        if model_output_classes == 2:
            self.last_dense = layers.Dense(units=1,
                                           activation="sigmoid")
        else:
            self.last_dense = layers.Dense(units=model_output_classes,
                                           activation="softmax")
    
    def call(self, inputs, training):
        l = self.embedding(inputs)
        l_1 = self.cnn_layer1(l) 
        l_1 = self.pool(l_1) 
        l_2 = self.cnn_layer2(l) 
        l_2 = self.pool(l_2)
        l_3 = self.cnn_layer3(l)
        l_3 = self.pool(l_3) 
        
        concatenated = tf.concat([l_1, l_2, l_3], axis=-1) # (batch_size, 3 * cnn_filters)
        concatenated = self.dense_1(concatenated)
        concatenated = self.dropout(concatenated, training)
        model_output = self.last_dense(concatenated)
        
        return model_output

In [0]:
"""
The three convolutional neural network layers are concatenated together and 
their output is fed to the first densely connected neural network.
The second densely connected neural network is used to predict the output sentiment since it only contains 2 classes. 
"""


VOCAB_LENGTH = len(tokenizer.vocab)
EMB_DIM = 200
CNN_FILTERS = 100
DNN_UNITS = 256
OUTPUT_CLASSES = 2

DROPOUT_RATE = 0.2

NB_EPOCHS = 10

In [0]:
text_model = TEXT_MODEL(vocabulary_size=VOCAB_LENGTH,
                        embedding_dimensions=EMB_DIM,
                        cnn_filters=CNN_FILTERS,
                        dnn_units=DNN_UNITS,
                        model_output_classes=OUTPUT_CLASSES,
                        dropout_rate=DROPOUT_RATE)

In [0]:
if OUTPUT_CLASSES == 2:
    text_model.compile(loss="binary_crossentropy",
                       optimizer="adam",
                       metrics=["accuracy"])
else:
    text_model.compile(loss="sparse_categorical_crossentropy",
                       optimizer="adam",
                       metrics=["sparse_categorical_accuracy"])

In [0]:
text_model.fit(train_data, epochs=NB_EPOCHS)

Epoch 1/10
650/650 [==============================] - 50s 77ms/step - loss: 0.2960 - accuracy: 0.8949
Epoch 2/10
650/650 [==============================] - 50s 76ms/step - loss: 0.1938 - accuracy: 0.9317
Epoch 3/10
650/650 [==============================] - 49s 76ms/step - loss: 0.0727 - accuracy: 0.9767
Epoch 4/10
650/650 [==============================] - 49s 76ms/step - loss: 0.0310 - accuracy: 0.9905
Epoch 5/10
650/650 [==============================] - 50s 76ms/step - loss: 0.0239 - accuracy: 0.9919
Epoch 6/10
650/650 [==============================] - 49s 76ms/step - loss: 0.0203 - accuracy: 0.9923
Epoch 7/10
650/650 [==============================] - 50s 76ms/step - loss: 0.0144 - accuracy: 0.9944
Epoch 8/10
650/650 [==============================] - 49s 75ms/step - loss: 0.0123 - accuracy: 0.9948
Epoch 9/10
650/650 [==============================] - 49s 75ms/step - loss: 0.0086 - accuracy: 0.9962
Epoch 10/10
650/650 [==============================] - 49s 76ms/step - loss: 0.008

In [0]:
results = text_model.evaluate(test_data)
print(results)

162/162 [==============================] - 2s 12ms/step - loss: 0.0026 - accuracy: 0.9986
[0.0025661408435553312, 0.998641848564148]
